<a href="https://colab.research.google.com/github/Hafsa-muhammad/project-1/blob/main/DataPreProcess_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install kaggle
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hafsaesam","key":"6a5ba5c25065ad86512dbebbaa1e58bc"}'}

In [ ]:
#Make directory name kaggle
! mkdir ~/.kaggle

In [ ]:
#Copy the json kaggle to this directory
! cp kaggle.json ~/.kaggle/


In [ ]:
#Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset                Hotel Reservations Dataset                          480KB  2023-01-04 12:50:31           8014        280  1.0              
googleai/musiccaps                                               MusicCaps                                           793KB  2023-01-25 09:25:48           1307        143  0.9411765        
themrityunjaypathak/most-subscribed-1000-youtube-channels        Most Subscribed 1000 Youtube Channels                28KB  2023-01-21 14:42:05           1766         57  1.0              
senapatirajesh/netflix-tv-shows-and-movies             

In [ ]:
!kaggle datasets download -d ranatalaat12/india-splitdataset

100% 401M/401M [00:21<00:00, 23.6MB/s]
100% 401M/401M [00:21<00:00, 19.6MB/s]


In [ ]:
# Put on the same directory
from zipfile import ZipFile 
  
# specifying the name of the zip file
file = "india-splitdataset.zip"
  
# open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # list all the contents of the zip file
    zip.printdir() 
  
    # extract all files
    print('extraction...') 
    zip.extractall() 
    print('Done!')

Streaming output truncated to the last 5000 lines.
split_data/train/images/India_006374.jpg       2023-02-02 11:08:26       107203
split_data/train/images/India_006375.jpg       2023-02-02 11:08:26        67071
split_data/train/images/India_006376.jpg       2023-02-02 11:08:26        67857
split_data/train/images/India_006381.jpg       2023-02-02 11:08:26        67608
split_data/train/images/India_006383.jpg       2023-02-02 11:08:26        67102
split_data/train/images/India_006386.jpg       2023-02-02 11:08:26        41960
split_data/train/images/India_006390.jpg       2023-02-02 11:08:26        44692
split_data/train/images/India_006392.jpg       2023-02-02 11:08:26        66396
split_data/train/images/India_006396.jpg       2023-02-02 11:08:26        85002
split_data/train/images/India_006398.jpg       2023-02-02 11:08:26        63897
split_data/train/images/India_006401.jpg       2023-02-02 11:08:26        45553
split_data/train/images/India_006402.jpg       2023-02-02 11:08:26   

In [ ]:
cd /content/split_data/

/content/split_data


In [ ]:
! pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json 
import numpy as np  
import cv2 
import os 
import xmltodict

In [ ]:
path_train_annotation = r"/content/split_data/train/annotations/xmls"
file_names = os.listdir(path_train_annotation) 
classes = {'D20':0 , 'D00': 2 , 'D40': 3 , 'D10' : 6,
           'D11' : 7 , 'D01'  : 8, 'D43' : 9 , 'D44' : 10} 
for item in file_names: 
    with open(os.path.join(path_train_annotation  , item ), 'r') as file_x: 
        dictionary = xmltodict.parse(file_x.read())
        name_txt_file = item.split('.')[0] + '.txt' 
        detections = dictionary['annotation']
        if isinstance(detections, dict): 
            detections = [detections]
        elif len(detections) == 0 : 
            # Write empty txt file
            print('Empty txt file')
            pass 
       
        # for detection in detections:
        #     class_name = classes[detection['annotation']['object']['name']]
        #     print(class_name)
        
        print(detections)
        #print(detections['annotation']['object'])

Streaming output truncated to the last 5000 lines.
[{'folder': 'images', 'filename': 'India_006884.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': {'name': 'D44', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '447', 'ymin': '461', 'xmax': '482', 'ymax': '581'}}}]
[{'folder': 'images', 'filename': 'India_009761.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0'}]
[{'folder': 'images', 'filename': 'India_009641.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': {'name': 'D20', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '503', 'ymin': '488', 'xmax': '714', 'ymax': '596'}}}]
[{'folder': 'images', 'filename': 'India_007243.jpg', 'size': {'depth': '3', 'width': '720', 'height': '720'}}]
[{'folder': 'images', 'filename': 'India_005259.jpg', 'size': {'depth': '3', 'width': '720', 'height': '720'}}]
[{'folder': 'images

**Making Labels** for train dataset **

In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "/content/split_data/train/annotations/xmls"
output_dir = "/content/split_data/train/labels/"
image_dir = "/content/split_data/train/images"

# create the labels folder (output directory)
os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

**Reading XMLS files in Validation data**

In [ ]:
path_valid_annotation = r"/content/split_data/val/annotations/xmls"
file_names = os.listdir(path_valid_annotation) 
classes = {'D20':0 , 'D00': 2 , 'D40': 3 , 'D10' : 6,
           'D11' : 7 , 'D01'  : 8, 'D43' : 9 , 'D44' : 10} 
for item in file_names: 
    with open(os.path.join(path_valid_annotation  , item ), 'r') as file_x: 
        dictionary_valid = xmltodict.parse(file_x.read())
        name_txt_file = item.split('.')[0] + '.txt' 
        detections_valid = dictionary_valid['annotation']
        if isinstance(detections_valid, dict): 
            detections_valid = [detections_valid]
        elif len(detections_valid) == 0 : 
            # Write empty txt file
            print('Empty txt file')
            pass 
        print(detections_valid)
        #print(detections['annotation']['object'])

[{'folder': 'images', 'filename': 'India_004749.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': {'name': 'D40', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '19', 'ymin': '438', 'xmax': '417', 'ymax': '695'}}}]
[{'folder': 'images', 'filename': 'India_000018.jpg', 'size': {'depth': '3', 'width': '720', 'height': '720'}}]
[{'folder': 'images', 'filename': 'India_009148.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'D40', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '316', 'ymin': '545', 'xmax': '398', 'ymax': '574'}}, {'name': 'D40', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '104', 'ymin': '620', 'xmax': '169', 'ymax': '659'}}, {'name': 'D40', 'pose': 'Unspecified', 'truncated': '1', 'difficult': '0', 'bndbox': {'xmin': '1', 'ymin': '550', 'xmax': '162', 'ymax': '665'}}, {'name': 'D40', 'po

**Making Labels for validation Data**

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "/content/split_data/val/annotations/xmls"
output_dir = "/content/split_data/val/labels/"
image_dir = "/content/split_data/val/images"

# create the labels folder (output directory)
os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

**Reading XMLS files from test data**

In [ ]:
path_test_annotation = r"/content/split_data/test/annotations/xmls"
file_names = os.listdir(path_test_annotation) 
classes = {'D20':0 , 'D00': 2 , 'D40': 3 , 'D10' : 6,
           'D11' : 7 , 'D01'  : 8, 'D43' : 9 , 'D44' : 10} 
for item in file_names: 
    with open(os.path.join(path_test_annotation  , item ), 'r') as file_x: 
        dictionary_test = xmltodict.parse(file_x.read())
        name_txt_file = item.split('.')[0] + '.txt' 
        detections_test = dictionary_test['annotation']
        if isinstance(detections_test, dict): 
            detections_test = [detections_test]
        elif len(detections_test) == 0 : 
            # Write empty txt file
            print('Empty txt file')
            pass 
        print(detections_test)

[{'folder': 'images', 'filename': 'India_005453.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'D00', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '527', 'ymin': '485', 'xmax': '603', 'ymax': '538'}}, {'name': 'D00', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '609', 'ymin': '539', 'xmax': '685', 'ymax': '592'}}]}]
[{'folder': 'images', 'filename': 'India_003577.jpg', 'size': {'depth': '3', 'width': '720', 'height': '720'}}]
[{'folder': 'images', 'filename': 'India_007204.jpg', 'size': {'width': '720', 'height': '720', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'D40', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '182', 'ymin': '498', 'xmax': '220', 'ymax': '514'}}, {'name': 'D40', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '217', 'ymin': '483', 'xmax': '255', 'ymax': '499'}}, {'name': 'D20'

In [ ]:
!rm -rf "/content/split_data/test/labels/"

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "/content/split_data/test/annotations/xmls"
output_dir = "/content/split_data/test/labels/"
image_dir = "/content/split_data/test/images"

# create the labels folder (output directory)
os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    # if root.find('object') is None:
    #     with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
    #       f.write("".join(result))

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
     

    if (result) or (root.find('object') is None):
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

**bounding box**

In [ ]:
#coding:utf-8
import cv2
import os
import random


# Global variables for path configuration
label_folder = './labels/'  #Detection result storage folder labels path

raw_images_folder = '/content/split_data/train/images' #Check the image storage folder raw_images path

save_images_folder = '/content/split_data/train/save_images'  #Save the picture folder save_image path

name_list_path = './name_list.txt' #There is a detection picture name txt file path

classes_path = './classes.txt'


def plot_one_box(x, image, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(image, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

#函数：在一幅图片对应位置上加上矩形框  image_name 图片名称不含后缀 
def draw_box_on_image(image_name, classes, colors, label_folder, raw_images_folder, save_images_folder ):
    txt_path  = os.path.join(label_folder,'%s.txt'%(image_name))  #本次检测结果txt路径
    print(image_name)
    if image_name == '.DS_Store':
        return 0
    image_path = os.path.join( raw_images_folder,'%s.jpg'%(image_name))  #本次原始图片jpg路径
    
    save_file_path = os.path.join(save_images_folder,'%s.jpg'%(image_name)) #本次保存图片jpg路径
    
    # flag_people_or_car_data = 0  #变量 代表类别
    source_file = open(txt_path) if os.path.exists(txt_path) else []
    image = cv2.imread(image_path)
    try:
        height, width, channels = image.shape
    except:
        print('no shape info.')
        return 0

    box_number = 0
    for line in source_file: #例遍 txt文件得每一行
        staff = line.split() #对每行内容 通过以空格为分隔符对字符串进行切片
        class_idx = int(staff[0])

        x_center, y_center, w, h = float(staff[1])*width, float(staff[2])*height, float(staff[3])*width, float(staff[4])*height
        x1 = round(x_center-w/2)
        y1 = round(y_center-h/2)
        x2 = round(x_center+w/2)
        y2 = round(y_center+h/2)     
        
        # if class_idx == 0: 
        #     draw_people_tangle = cv2.rectangle(image, (x1,y1),(x2,y2),(0,0,255),2)   # 画框操作  红框  宽度为1
        #     cv2.imwrite(save_file_path,draw_people_tangle)  #画框 并保存
        # elif class_idx == 1:
        #     draw_car_tangle = cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),2)     # 画框操作  绿框  宽度为1
        #     cv2.imwrite(save_file_path,draw_car_tangle)  #画框 并保存

        plot_one_box([x1,y1,x2,y2], image, color=colors[class_idx], label=classes[class_idx], line_thickness=None)

        cv2.imwrite(save_file_path,image) 

        box_number += 1
    return box_number
    

#函数：通过保存有原始图片得文件夹，生成写有所有检测图片名称（不带后缀）得txt
def make_name_list(raw_images_folder, name_list_path):

    image_file_list = os.listdir(raw_images_folder) #得到该路径下所有文件名称带后缀

    text_image_name_list_file=open(name_list_path,'w')  #以写入的方式打开txt ，方便更新 不要用追加写

    for  image_file_name in image_file_list:#例遍写入
        image_name,file_extend = os.path.splitext(image_file_name)  # 去掉扩展名
        text_image_name_list_file.write(image_name+'\n') #写入
    
    text_image_name_list_file.close()


if __name__ == '__main__':           # 只有在文件作为脚本文件直接执行时才执行下面代码  

    make_name_list(raw_images_folder, name_list_path) #执行写入txt函数

    classes = image_names = open(classes_path).read().strip().split('\n')
    random.seed(42)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]

    image_names = open(name_list_path).read().strip().split() #得到图片名字不带后缀

    box_total = 0
    image_total = 0
    for image_name in image_names: #例遍图片名称
        box_num = draw_box_on_image(image_name, classes, colors, label_folder, raw_images_folder, save_images_folder)#对图片画框
        box_total += box_num
        image_total += 1
        print('Box number:', box_total, 'Image number:',image_total)

Streaming output truncated to the last 5000 lines.
India_000002
Box number: 0 Image number: 2895
India_006464
Box number: 0 Image number: 2896
India_001267
Box number: 0 Image number: 2897
India_009720
Box number: 0 Image number: 2898
India_007736
Box number: 0 Image number: 2899
India_004027
Box number: 0 Image number: 2900
India_007811
Box number: 0 Image number: 2901
India_005521
Box number: 0 Image number: 2902
India_003773
Box number: 0 Image number: 2903
India_003460
Box number: 0 Image number: 2904
India_001348
Box number: 0 Image number: 2905
India_004956
Box number: 0 Image number: 2906
India_009579
Box number: 0 Image number: 2907
India_005223
Box number: 0 Image number: 2908
India_003384
Box number: 0 Image number: 2909
India_002173
Box number: 0 Image number: 2910
India_003917
Box number: 0 Image number: 2911
India_001331
Box number: 0 Image number: 2912
India_001966
Box number: 0 Image number: 2913
India_008616
Box number: 0 Image number: 2914
India_006161
Box number: 0 Im